In [ ]:
import openai
from statistics import mean
from math import exp
import pandas as pd

openai.api_key = pd.read_csv("prod_env/openai_api_key.csv", header=None).values[0][0]

In [ ]:
NUM_COMPLETIONS = 1
MAX_TOKENS = 10
TEMPERATURE = 1
MODEL_NAME = "openai/text-davinci-003"
TOP_K_PER_TOKEN = 5
ECHO_PROMPT = False

STOP_SEQUENCES = []

In [ ]:
folder_path = '/u/scr/katezhou/Uncertainty/interpretability/'

prefix_df = pd.read_csv(folder_path + "uncertainty_template_meta.csv")
PREFIXES = [""] + [" " + x for x in  prefix_df['name'].values]
PREFIXES.remove(" unprompted")
PREFIXES.remove( " I don't think it's")
PREFIXES.remove(" I seriously doubt it's")
print(len(PREFIXES))

prefixes_df = pd.DataFrame()
prefixes_df['prefix']=PREFIXES

In [ ]:
ppls = []
for n, row in prefix_df.iterrows():
    prompt=row['name']
    if prompt=="unprompted":
        ppls.append(0)
        continue
    completion = openai.Completion.create(model="davinci", 
                                          prompt=prompt,
                                         temperature=1,
                                         logprobs=5,
                                         max_tokens=0,
                                         echo=True)
    nll = []
    nll.append(mean([i for i in completion['choices'][0]['logprobs']['token_logprobs'] if i is not None]))
    ppl = [exp(-i) for i in nll]
    ppls.append(ppl[0])
prefix_df['ppl'] = ppls

In [ ]:
# pd.read_csv("uncertainty_template_meta_perplexity_davinci.csv").sort_values(by='ppl')

In [ ]:
prefix_df.sort_values(by='ppl')